In [1]:
import os
import cv2
from PIL import Image
import pandas as pd

In [2]:
import base64
from io import BytesIO

def img_2_b64(image):
    buff = BytesIO()
    image.save(buff, format="JPEG")
#    img_str = base64.b64encode(buff.getvalue())
    img_str = buff.getvalue()

    return img_str

# Convert Base64 to Image
def b64_2_img(data):
#    buff = BytesIO(base64.b64decode(data))
    buff = BytesIO(data)

    return Image.open(buff)

In [3]:
def load_images_from_directory(src):
    TARGET_SIZE = (223, 223)

    data = dict()
    data['labels'] = []
    data['images'] = []
    data['resized'] = []
    data['filename'] = []

    for root, subdir, files in os.walk(src):
        for class_name in subdir:
            class_dir = os.path.join(root, class_name)
            for filename in os.listdir(class_dir):
                img_path = os.path.join(class_dir, filename)
                with Image.open(img_path) as img:
                    resized_img = img
                    data['labels'].append(class_name)
                    data['resized'].append(img_2_b64(resized_img.resize(TARGET_SIZE, Image.Resampling.LANCZOS)))
                    data['images'].append(img_2_b64(img))
                    data['filename'].append(filename)
    return data

In [4]:
directory   = fr'.\FAITH-Bark\Tree'
dataset = load_images_from_directory(directory)

In [5]:
import matplotlib.pyplot as plt

In [6]:
def visualize(image, transform=None, x=200, y=200):
    plt.figure(figsize=(x, y))
    plt.axis('off')
    plt.imshow(image)

In [8]:
image_df = pd.DataFrame.from_dict(dataset, orient='index').transpose()
os.makedirs(r'.\out', exist_ok=True)
image_df.to_parquet(path=r'.\out/dataset.parquet')